## 09 - Multi-Agent Systems (Supervisor Pattern)

**Key Concept**: Complex applications often require multiple specialized agents working together. The supervisor pattern uses a central controller agent to coordinate specialized worker agents, each handling a distinct domain.

**What this covers:**
1. Why multi-agent architectures matter
2. Tool calling pattern: agents as tools
3. Building specialized sub-agents
4. Creating a supervisor agent
5. Controlling information flow between agents
6. Adding human-in-the-loop review

from langchain_groq import ChatGroq
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime, InjectedToolCallId
from langchain.agents import AgentState
from langchain_core.messages import ToolMessage
from langgraph.types import Command

from pydantic import BaseModel, Field
from typing import Annotated, Optional, List, Literal
from datetime import datetime

In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,  # Deterministic for consistent multi-agent coordination
)


### Why Multi-Agent Systems?

Single agents struggle when:
- Too many tools lead to poor tool selection
- Context grows too large for one agent to track
- Tasks require distinct specializations (planner, researcher, executor)

**Two main patterns:**

| Pattern | How it works | Control flow |
|---------|--------------|--------------|
| **Tool Calling** | Supervisor treats sub-agents as tools. Sub-agents run tasks and return results. | Centralized: all routing through supervisor |
| **Handoffs** | Current agent transfers control to another. Active agent changes dynamically. | Decentralized: agents can switch who's active |

This notebook focuses on **Tool Calling** (supervisor pattern) - the more common choice for structured workflows.

### Step 1: Define Domain-Specific Tools

Before creating sub-agents, we define the tools each specialist will use.
We'll build a personal assistant with:
- **Calendar agent**: scheduling, availability checking
- **Email agent**: drafting and sending messages

In [2]:
from langchain.tools import tool
# Calendar tools - simulated for demonstration
@tool
def check_calendar(date: str) -> str:
    """Check calendar availability for a specific date."""
    # Simulated calendar data
    busy_slots = {
        "2024-12-05": ["10:00-11:00 Team standup", "14:00-15:00 Client call"],
        "2024-12-06": ["09:00-10:00 Planning meeting"],
    }
    events = busy_slots.get(date, [])
    if events:
        return f"Events on {date}: {', '.join(events)}"
    return f"No events scheduled for {date}. Day is free."

@tool
def schedule_meeting(title: str, date: str, time: str, attendees: str) -> str:
    """Schedule a new meeting on the calendar."""
    return f"Meeting scheduled: '{title}' on {date} at {time} with {attendees}"

@tool
def get_upcoming_events(days: int = 7) -> str:
    """Get upcoming calendar events for the next N days."""
    return f"Upcoming events (next {days} days): Team standup (Dec 5), Client call (Dec 5), Planning (Dec 6)"

calendar_tools = [check_calendar, schedule_meeting, get_upcoming_events]
print(f"Calendar tools: {[t.name for t in calendar_tools]}")


Calendar tools: ['check_calendar', 'schedule_meeting', 'get_upcoming_events']


In [3]:
# Email tools - simulated for demonstration
@tool
def search_emails(query: str) -> str:
    """Search emails by keyword or sender."""
    # Simulated email search
    emails = {
        "project": "Found 3 emails about 'project': Project Update (Dec 3), Project Review (Dec 1), Project Kickoff (Nov 28)",
        "meeting": "Found 2 emails about 'meeting': Meeting Invite (Dec 4), Meeting Notes (Dec 2)",
    }
    for key, result in emails.items():
        if key in query.lower():
            return result
    return f"No emails found matching '{query}'"

@tool
def draft_email(to: str, subject: str, body: str) -> str:
    """Draft an email (does not send)."""
    return f"Draft created:\nTo: {to}\nSubject: {subject}\nBody: {body[:100]}..."

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to the specified recipient."""
    return f"Email sent to {to} with subject '{subject}'"

email_tools = [search_emails, draft_email, send_email]
print(f"Email tools: {[t.name for t in email_tools]}")


Email tools: ['search_emails', 'draft_email', 'send_email']


### Step 2: Create Specialized Sub-Agents

Each sub-agent is a standalone agent with:
- Its own tools (domain-specific)
- A focused system prompt (expertise area)
- Independent reasoning capability

The key insight: sub-agents don't need to know about other sub-agents. They just do their job well.


In [6]:
# Calendar sub-agent: handles all scheduling-related tasks
from langchain.agents import create_agent

calendar_agent = create_agent(
    llm,
    tools=calendar_tools,
    system_prompt="""You are a calendar management specialist. Your job is to:
- Check availability on specific dates
- Schedule meetings and events
- Retrieve upcoming events

Always confirm what action you took. Include all relevant details (date, time, attendees) 
in your final response - the supervisor only sees your final message."""
)

print("Calendar agent created")


Calendar agent created


In [7]:
# Email sub-agent: handles all communication tasks
email_agent = create_agent(
    llm,
    tools=email_tools,
    system_prompt="""You are an email management specialist. Your job is to:
- Search through emails
- Draft professional emails
- Send emails when requested

Be concise but professional in drafted emails. Always confirm what action you took
in your final response - the supervisor only sees your final message."""
)

print("Email agent created")


Email agent created


In [8]:
# Test sub-agents independently before wiring them together
# This is good practice - debug individual agents before composing them

result = calendar_agent.invoke({
    "messages": [{"role": "user", "content": "What's on my calendar for 2024-12-05?"}]
})
print("Calendar agent response:")
print(result["messages"][-1].content)


Calendar agent response:
I checked your calendar for 2024‑12‑05. You have two events scheduled:

- **10:00 – 11:00** – Team standup  
- **14:00 – 15:00** – Client call  

Let me know if you’d like to add, move, or cancel any of these.


### Step 3: Wrap Sub-Agents as Tools

The core of the supervisor pattern: sub-agents become tools that the supervisor can call.

**Why this works:**
- Supervisor sees sub-agents as capabilities (tools), not implementation details
- Each tool has a clear name and description guiding when to use it
- Sub-agent complexity is hidden from the supervisor


In [21]:
# Basic pattern: wrap sub-agent as a tool
# The tool name and description are critical - they guide the supervisor's decisions

@tool("manage_calendar", description="Handle calendar operations: check availability, schedule meetings, view upcoming events")
def manage_calendar(request: str) -> str:
    """Route calendar-related requests to the calendar specialist."""
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    # Return the final message content - this is what supervisor sees
    return result["messages"][-1].content

@tool("manage_email", description="Handle email operations: search emails, draft messages, send emails")
def manage_email(request: str) -> str:
    """Route email-related requests to the email specialist."""
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].content

supervisor_tools = [manage_calendar, manage_email]
print(f"Supervisor tools: {[t.name for t in supervisor_tools]}")


Supervisor tools: ['manage_calendar', 'manage_email']


### Step 4: Create the Supervisor Agent

The supervisor orchestrates the workflow:
1. Receives user request
2. Decides which sub-agent(s) to call
3. Synthesizes results into a coherent response

The supervisor doesn't need domain-specific tools - it just needs to know how to delegate.


In [10]:
# Create the supervisor agent
supervisor = create_agent(
    llm,
    tools=supervisor_tools,
    system_prompt="""You are a personal assistant that coordinates specialized agents.

You have access to:
- manage_calendar: For scheduling, availability, and calendar queries
- manage_email: For email search, drafting, and sending

For each user request:
1. Identify which specialist(s) are needed
2. Delegate specific tasks to the appropriate specialist
3. Combine results into a helpful response

Be concise. If a task spans multiple domains, call multiple specialists."""
)

print("Supervisor agent created")


Supervisor agent created


### Step 5: Use the Supervisor

Let's test the supervisor with different request types:
1. Single-domain request (calendar only)
2. Multi-domain request (calendar + email)


In [11]:
# Example 1: Single-domain request
# Supervisor should route this to calendar agent only

result = supervisor.invoke({
    "messages": [{"role": "user", "content": "What meetings do I have coming up this week?"}]
})

print("Single-domain request:")
print("-" * 40)
print(result["messages"][-1].pretty_print())


Single-domain request:
----------------------------------------
================================== Ai Message ==================================

Here are the meetings you have scheduled for the upcoming week:

- **Team standup** – December 5  
- **Client call** – December 5  
- **Planning** – December 6  

No other events are on your calendar for this period. Let me know if you’d like to add, move, or cancel any of them.
None


In [12]:
# Example 2: Multi-domain request
# Supervisor should coordinate both calendar and email agents

result = supervisor.invoke({
    "messages": [{
        "role": "user", 
        "content": "Check my calendar for December 5th and send an email to team@company.com about the meeting schedule"
    }]
})

print("Multi-domain request:")
print("-" * 40)
print(result["messages"][-1].pretty_print())


Multi-domain request:
----------------------------------------
================================== Ai Message ==================================

Your calendar on **December 5, 2025** is completely free, and the email to **team@company.com** with the subject **“Meeting Schedule”** has been sent. Let me know if you need anything else!
None


### Understanding the Architecture

```
User Request
     |
     v
+------------+
| Supervisor |  <-- Decides routing based on request
+------------+
     |
     +---> manage_calendar (tool) ---> Calendar Agent ---> Calendar Tools
     |
     +---> manage_email (tool) ---> Email Agent ---> Email Tools
     |
     v
Final Response (synthesized by supervisor)
```

**Key points:**
- Each layer has a single responsibility
- Sub-agents are isolated - they don't know about each other
- Supervisor handles orchestration logic


### Advanced: Control Information Flow

By default, sub-agents only see the request string passed by the supervisor.
Sometimes you need more control:
- **Input control**: Pass conversation history or state to sub-agents
- **Output control**: Return structured data, not just text


In [13]:
# Custom state to share between supervisor and sub-agents
from langchain.agents import AgentState
from langchain.tools import tool, ToolRuntime

class AssistantState(AgentState):
    """Extended state that tracks additional context."""
    user_timezone: str = "UTC"
    priority_level: str = "normal"

# Context-aware tool: injects conversation context into sub-agent
# ToolRuntime[ConfigType, StateType] When to use None: When your tool doesn't need runtime configuration
@tool("manage_calendar_with_context", description="Calendar operations with full conversation context")
def manage_calendar_with_context(
    request: str, 
    runtime: ToolRuntime[None, AssistantState]
) -> str:
    """Route to calendar agent with additional context from conversation."""
    # Get the original user message for context
    messages = runtime.state.get("messages", [])
    original_request = next(
        (msg.content for msg in messages if hasattr(msg, "content") and msg.type == "human"),
        request
    )
    
    # Build enriched prompt with context
    enriched_prompt = f"""User's original request: {original_request}

Current task: {request}
User timezone: {runtime.state.get('user_timezone', 'UTC')}
Priority: {runtime.state.get('priority_level', 'normal')}"""
    
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": enriched_prompt}]
    })
    return result["messages"][-1].content

print("Context-aware calendar tool created")


Context-aware calendar tool created


In [14]:
# Control output: return structured data using Command
# This allows sub-agents to update shared state, not just return text

from typing import Annotated

from langchain.tools import InjectedToolCallId
from langchain_core.messages import ToolMessage
from langgraph.types import Command


@tool("manage_email_structured", description="Email operations that return structured results")
def manage_email_structured(
    request: str,
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Route to email agent and return structured response with state updates."""
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    
    final_content = result["messages"][-1].content
    
    # Determine if this was a send action (for tracking)
    action_type = "send" if "sent" in final_content.lower() else "other"
    
    # Return Command to update state alongside the tool response
    return Command(update={
        "messages": [
            ToolMessage(
                content=final_content,
                tool_call_id=tool_call_id
            )
        ],
        # Could add custom state updates here
        # "last_email_action": action_type,
    })

print("Structured email tool created")


Structured email tool created


### Human-in-the-Loop Review

For sensitive actions (sending emails, scheduling meetings), you may want human approval.

**Key insight**: Apply `HumanInTheLoopMiddleware` to the **sub-agents**, not the supervisor.
The middleware intercepts tool calls within sub-agents (like `send_email`), and interrupts bubble up through the supervisor to the user.


In [24]:
# Step 1: Create sub-agents WITH human-in-the-loop middleware
# The middleware is applied to sub-agents, intercepting their sensitive tool calls

from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import MemorySaver

# Checkpointer is required for interrupt functionality
checkpointer = MemorySaver()

# Calendar agent with HITL on schedule_meeting
calendar_agent_hitl = create_agent(
    llm,
    tools=calendar_tools,
    system_prompt="""You are a calendar management specialist. Your job is to:
- Check availability on specific dates
- Schedule meetings and events
- Retrieve upcoming events

Always confirm what action you took in your final response.""",
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"schedule_meeting": True},  # Pause before scheduling
            description_prefix="Calendar event pending approval",
        ),
    ],
)

# Email agent with HITL on send_email
email_agent_hitl = create_agent(
    llm,
    tools=email_tools,
    system_prompt="""You are an email management specialist. Your job is to:
- Search through emails
- Draft professional emails
- Send emails when requested

Always confirm what action you took in your final response.""",
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"send_email": True},  # Pause before sending
            description_prefix="Outbound email pending approval",
        ),
    ],
)

print("Sub-agents with human-in-the-loop created")


Sub-agents with human-in-the-loop created


In [25]:
# Step 2: Wrap HITL sub-agents as tools for the supervisor
# These tools now route to agents that can interrupt for approval

@tool("schedule_event", description="Schedule calendar events. Will pause for human approval before creating.")
def schedule_event(request: str) -> str:
    """Route scheduling requests to calendar agent with HITL."""
    result = calendar_agent_hitl.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].content

@tool("manage_email_hitl", description="Handle email operations. Will pause for human approval before sending.")
def manage_email_hitl(request: str) -> str:
    """Route email requests to email agent with HITL."""
    result = email_agent_hitl.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].content

hitl_tools = [schedule_event, manage_email_hitl]
print(f"HITL supervisor tools: {[t.name for t in hitl_tools]}")


HITL supervisor tools: ['schedule_event', 'manage_email_hitl']


In [26]:
# Step 3: Create supervisor with checkpointer (required for interrupts to bubble up)

supervisor_with_review = create_agent(
    llm,
    tools=hitl_tools,
    system_prompt="""You are a personal assistant coordinating calendar and email specialists.

- schedule_event: For scheduling meetings (requires approval)
- manage_email_hitl: For email operations (requires approval before sending)

Delegate tasks to the appropriate specialist.""",
    checkpointer=checkpointer,
)

print("Supervisor with HITL sub-agents created")


Supervisor with HITL sub-agents created


In [27]:
# Demonstrate the interrupt flow
# Request spans both domains - will trigger interrupts for both schedule and send

config = {"configurable": {"thread_id": "hitl-demo-1"}}

query = "Schedule a meeting with the design team next Tuesday at 2pm and send them an email reminder about reviewing mockups"


interrupts = []
for step in supervisor_with_review.stream(
    {"messages": [{"role": "user", "content": query}]},
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            # Interrupt occurred
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\n>>> INTERRUPTED: {interrupt_.id}")
            

================================== Ai Message ==================================
Tool Calls:
  schedule_event (fc_287e1e10-4894-436c-ad5d-e4ec9782fc60)
 Call ID: fc_287e1e10-4894-436c-ad5d-e4ec9782fc60
  Args:
    request: Schedule a meeting with the design team next Tuesday at 2pm.

>>> INTERRUPTED: 11d7ca947fc8bb77596bad9de631fef5


In [ ]:
# Inspect what's pending approval
# Each interrupt contains action_requests with tool name and arguments

for interrupt_ in interrupts:
    for request in interrupt_.value["action_requests"]:
        print(f"INTERRUPT ID: {interrupt_.id}")
        print(f"{request['description']}\n")


INTERRUPT ID: 11d7ca947fc8bb77596bad9de631fef5
Calendar event pending approval

Tool: schedule_meeting
Args: {'attendees': 'design team', 'date': '2025-12-09', 'time': '14:00', 'title': 'Design Team Meeting'}



In [29]:
# Resume with decisions: approve, reject, or edit each pending action
# Using Command to specify decisions for each interrupt by ID

from langgraph.types import Command

resume = {}
for interrupt_ in interrupts:
    # For demo: approve all pending actions
    # In production, you'd inspect and decide per action
    resume[interrupt_.id] = {"decisions": [{"type": "approve"}]}

# Alternative: edit an action before approving
# edited_action = interrupt_.value["action_requests"][0].copy()
# edited_action["arguments"]["subject"] = "Updated subject line"
# resume[interrupt_.id] = {"decisions": [{"type": "edit", "edited_action": edited_action}]}

# Alternative: reject an action
# resume[interrupt_.id] = {"decisions": [{"type": "reject"}]}

print("Decisions prepared:")
for interrupt_id, decision in resume.items():
    print(f"  {interrupt_id[:20]}... -> {decision['decisions'][0]['type']}")


Decisions prepared:
  11d7ca947fc8bb77596b... -> approve


In [30]:
# Resume execution with our decisions
# The run continues from where it was interrupted

for step in supervisor_with_review.stream(
    Command(resume=resume),
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            # Another interrupt (shouldn't happen if we approved all)
            print(f"\n>>> INTERRUPTED again: {update[0].id}")


================================= Tool Message =================================
Name: schedule_event

I scheduled the meeting titled **“Design Team Meeting”** for **Tuesday, December 9, 2025 at 2:00 PM** with the design team.
================================== Ai Message ==================================
Tool Calls:
  manage_email_hitl (fc_56375151-9abd-451b-af84-9d6d159e7a18)
 Call ID: fc_56375151-9abd-451b-af84-9d6d159e7a18
  Args:
    request: Send an email reminder to the design team about reviewing mockups for the meeting scheduled on Tuesday, December 9, 2025 at 2:00 PM. Include meeting details and ask them to review the mockups before the meeting.

>>> INTERRUPTED again: db4fe43a3c84d8021f191bed9eef0241


### HITL Decision Types

When resuming from an interrupt, you have three options:

| Decision | Effect |
|----------|--------|
| `{"type": "approve"}` | Execute the action as-is |
| `{"type": "reject"}` | Skip the action, continue workflow |
| `{"type": "edit", "edited_action": {...}}` | Modify arguments, then execute |

This gives fine-grained control over sensitive operations while keeping the workflow automated.

### When to Use What - From Documentation

| Scenario | Pattern |
|----------|---------|
| Structured workflow with clear domains | Supervisor (tool calling) |
| Agents need to talk to user directly | Handoffs |
| Need centralized control and logging | Supervisor |
| Complex multi-turn conversations per domain | Handoffs |
| Simple delegation with result synthesis | Supervisor |


### Key Takeaways

1. **Supervisor pattern** = sub-agents wrapped as tools, called by a central coordinator
2. **Clear boundaries**: each sub-agent has focused tools and prompts
3. **Information flow control**: customize what sub-agents see (input) and return (output)
4. **Human-in-the-loop**: apply `HumanInTheLoopMiddleware` to sub-agents, interrupts bubble up
5. **Test independently**: verify sub-agents work before composing them

For more complex scenarios where agents need to converse directly with users, explore the **handoffs** pattern.
